In [1]:
from SD_IB_IRP_PPenv import steroid_IRP
from Policies import policies
import matplotlib.pyplot as plt

# Policy Evaluation

This Notebook has a complete Policy Evaluation function for the Stochastic-Dynamic Inventory-Routing-Problem with Perishable Products. First, the main parameters of the problem and the environment must be set. All the main customizable parameters are below.

In [2]:
#################################   Environment's parameters   #################################
# SD-IB-IRP-PP model's parameters
horizon_type = 'episodic'
backorders = False
stochastic_parameters = False

# Feature's parameters
look_ahead = ['*']
historical_data = ['*']

# Action's parameters
validate_action = False
warnings = True

# Other parameters
rd_seed = 0
num_episodes = 1000
env_config = {  'M': 4, 
                'K': 4, 
                'T': 5, 
                'F': 10,
                
                'min_sprice': 1,
                'max_sprice': 500, 
                'min_hprice': 1,
                'max_hprice': 500,

                'back_o_cost':1000,
                
                'S': 5, 
                'LA_horizon': 4, 
                'lambda1': 0.5
            }
#################################   Environment's parameters   #################################

Next, the policy evaluation is defined. This function only takes one parameter, the number of episodes that will be runned of the environment. All the policies to be evaluated must be in the 'Policies.py' file. Any policy used must be able to receive the state, the additional information in _ and the environment as a parameters. Also, the policy function must return an action in the format defined in the Toying.ipynb.

In [7]:
'''

'''
def Policy_evaluation(num_episodes = 1000):
    
    rewards = {}
    states = {}
    real_actions = {}
    backorders = {}
    la_decisions = {}
    tws = {}
    env = steroid_IRP(  horizon_type = horizon_type, look_ahead = look_ahead, historical_data = historical_data,
                        backorders = backorders, stochastic_parameters = stochastic_parameters, rd_seed = rd_seed,
                        env_config = env_config)

    policy = policies()

    for episode in range(2):

        state, _ = env.reset(return_state = True)
        print(env.d)
        done = False

        while not done:
            
            states[episode,env.t] = state
            action, la_dec = policy.det_rolling_horizon(state, _, env)
            print(state)
            print(action)
            state, reward, done, real_action, _,  = env.step(action, validate_action = validate_action, warnings = warnings)

            real_actions[episode,env.t] = real_action
            backorders[episode,env.t] = _["bo"]
            rewards[episode,env.t] = reward
            la_decisions[episode,env.t] = la_dec
            tws[episode,env.t] = _["sample_path_window_size"]
            
    iterables = (env.Suppliers,env.Products,env.Samples,env.M_kt,env.O_k)
    costs = (env.c, env.h_t, env.p_t)

    return rewards, states, real_actions, backorders, la_decisions, tws, iterables, costs

In [ ]:
rewards, states, real_actions, backorders, la_decisions, tws, iterables = Policy_evaluation(num_episodes = num_episodes)

In [ ]:
def visualize_by_day(episode, day, states, real_actions, backorders, la_decisions, tws, iterables, costs):
    
    M, K, S, M_kt, O_k = iterables

    ''' Realized, Historical Decisions '''
    hist_T = range(0,day+1)
    # Initial Inventory level for product k, aged o
    ii_0 = {t:states[episode,t] for t in hist_T}
    # Routing Decisions
    rout = {t:real_actions[episode,t][0] for t in hist_T}
    # Purchasing decisions
    purch = {t:real_actions[episode,t][1] for t in hist_T}
    # Backorders
    back = {t:backorders[episode,t] for t in hist_T}

    ''' Look-ahead Decisions '''
    la_T = range(day,day+tws[episode,day])
    # Initial Inventory level for product k, aged o in sample path s
    ii_0_la = {(t):la_decisions[episode,day][0][t-day] for t in la_T}
    # Purchase Decisions
    purch_la = {(t):la_decisions[episode,day][1][t-day] for t in la_T}
    # Backorders
    back_la = {(t):la_decisions[episode,day][2][t-day] for t in la_T}

    ''' First Chart: Quantities '''

    spec = {"height_ratios":[1, 1, 1, 1],"hspace":0.25,"bottom":0.1,"top":0.9}
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(nrows=4,ncols=1,figsize=(13,20),gridspec_kw=spec)

    


    
    

